In [1]:
import os, pandas as pd
from pymongo import MongoClient
from dotenv import load_dotenv

load_dotenv()
client = MongoClient(os.getenv("MONGODB_URI"))
client.admin.command("ping")
db = client.sustainability_tracker
print("Connected")

Connected


In [3]:
CATS = ["Food","Goods"]
col = db["categories"]
for name in CATS:
    col.update_one({"name":name},
                   {"$setOnInsert":{"name":name,"description":f"{name} related emissions"}},
                   upsert=True)
list(db.categories.find({},{"_id":0}))


[{'name': 'Food', 'description': 'Food related emissions'},
 {'name': 'Transportation', 'description': 'Transportation related emissions'},
 {'name': 'Energy', 'description': 'Energy related emissions'},
 {'name': 'Goods', 'description': 'Goods related emissions'}]

In [4]:
food = pd.read_csv("data/food_production.csv")
item_col = "Food product"
ghg_col = "Total_emissions"  
rows=[]
for _,r in food.iterrows():
    item = str(r[item_col]).strip()
    if not item: continue
    co2e_per_kg = float(r[ghg_col])
    rows.append({
      "category_id": db.categories.find_one({"name":"Food"})["_id"],
      "item_name": item,
      "item_name_key": item.lower(),
      "co2e_per_unit": co2e_per_kg,
      "unit":"kg", "region":"Global","year":2018,"source":"Food production dataset"
    })
db.emission_factors.insert_many(rows)
len(rows)

43

In [24]:
epa = pd.read_csv("data/epa_emmisions.csv")
ITEM="2017 NAICS Title"
UNIT="Unit"
CO2E="Supply Chain Emission Factors with Margins"
rows=[]
for _,r in epa.iterrows():
    item=str(r[ITEM]).strip()
    if not item: continue
    unit=str(r[UNIT]).strip() if pd.notna(r[UNIT]) else "unit"
    co2e=float(r[CO2E]) if pd.notna(r[CO2E]) else None
    if co2e is None: continue
    rows.append({
      "category_id": db.categories.find_one({"name":"Goods"})["_id"],
      "item_name": item,
      "item_name_key": item.lower(),
      "co2e_per_unit": co2e,
      "unit": unit, "region":"US","year":2017,"source":"EPA supply-chain factors (USEEIO)"
    })
db.emission_factors.insert_many(rows)
len(rows)

1016

In [5]:
pipeline = [
    {
        "$lookup": {
            "from": "categories",
            "localField": "category_id",
            "foreignField": "_id",
            "as": "category"
        }
    },
    {
        "$unwind": "$category"
    },
    {
        "$group": {
            "_id": "$category.name",
            "count": { "$sum": 1 }
        }
    }
]

result = db.emission_factors.aggregate(pipeline)
for doc in result:
    print(doc)

{'_id': 'Food', 'count': 172}
{'_id': 'Goods', 'count': 1016}


In [7]:
print("collections:", db.list_collection_names())
print("factors:", db.emission_factors.count_documents({}))
print("sample food:", list(db.emission_factors.find({"unit":"kg"},{"_id":0,"item_name":1,"co2e_per_unit":1}).limit(3)))
print("sample epa:", list(db.emission_factors.find({"unit":{"$regex":"co2e/","$options":"i"}},{"_id":0,"item_name":1,"unit":1,"co2e_per_unit":1}).limit(3)))

collections: ['activities', 'emission_factors', 'users', 'categories']
factors: 1188
sample food: [{'item_name': 'Maize (Meal)', 'co2e_per_unit': 1.1}, {'item_name': 'Cassava', 'co2e_per_unit': 0.9}, {'item_name': 'Beet Sugar', 'co2e_per_unit': 1.4}]
sample epa: [{'item_name': 'Oilseed and Grain Combination Farming', 'co2e_per_unit': 0.848, 'unit': 'kg CO2e/2022 USD, purchaser price'}, {'item_name': 'Other Vegetable (except Potato) and Melon Farming', 'co2e_per_unit': 0.631, 'unit': 'kg CO2e/2022 USD, purchaser price'}, {'item_name': 'Orange Groves', 'co2e_per_unit': 0.488, 'unit': 'kg CO2e/2022 USD, purchaser price'}]


In [8]:
print("Total documents in emission_factors:", db.emission_factors.count_documents({}))

Total documents in emission_factors: 1188
